In [9]:
import regex as re

Task 1. 

Let's imagine a situation: you went to the market and filled your baskets (basket1 and basket2) with fruits. You wanted to have one of each kind but realized that some fruits were put in both baskets.

1. Your first task is to remove everything from basket2 that is already present in basket1.

In [3]:
def remove_dupl(basket1, basket2) :
    return [fruit for fruit in basket1 if not(fruit in basket2)]

In [4]:
remove_dupl([1, 2, 3], [2, 4, 5, 3])

[1]

2. After the removal it is reasonable to anticipate that one of the baskets might weigh more compared to the another (all fruit kinds weight the same). Therefore, the second task is to transfer some fruits from a heavier basket to the lighter one to get approximately the same weight/amount of fruits.

In [6]:
def balance(basket1, basket2) :
    basket = basket1 + basket2
    n = len(basket)//2
    return basket[:n], basket[n:]

In [7]:
balance([1, 2, 3, 4, 5], [6, 7])

([1, 2, 3], [4, 5, 6, 7])

Task 2. 

Your task is to create a regular expression matching a valid temperature represented either in Celsius or Fahrenheit scale (e.g. '+23.5 C', '-4 F', '0.0 C', '73.45 F') and to extract all the appearances from the given string text. Positive temperatures can be with or without the + prefix (e.g. '5 F', '+5 F'). Negative temperatures must be prefixed with -. Zero temperature can be used with a prefix or without.

In [12]:
def find_temp(text) :
    return [item[0] for item in re.findall('([+-]?\d?\d([.]\d\d?)? [CF])', text)]

In [13]:
find_temp("Hello. +23.5 C here. It's pretty hot. Better than last month's -21 F")

['+23.5 C', '-21 F']

Task 3.

You should create and use two tables:

calls : one row per client’s call. A client can call multiple times. When incoming_number is NULL, it means that the client called with a hidden phone number.

clients_crm : one row per client registered in our CRM. Client data is manually entered by agents, and then subject to errors and duplicates. CreationDate is not necessarily the date of the call, as a client can call multiple times and an agent can write down information on a paper and copy it later into the CRM. 

CREATE TABLE calls(
    id int,
    called_number varchar(15),
    date date,
    duration_in_sec int,
    incoming_number varchar(15));

CREATE TABLE clients_crm(
    id int,
    FirstName varchar(30),
    LastName varchar(30),
    PhoneNumber varchar(15),
    CreationDate date);

1. Perform a simple query to list the number of calls per day.

SELECT count(*) as num_calls FROM calls
GROUP BY DAY(date);

2. In our business, timing is key. A growth hacker needs to understand when clients are ﬁrst contacting us, even if ﬁrst contact does not lead to a client record inscription in our CRM. For that purpose, he asks you to provide a table with the following structure :

a.    client ID if available

b.    client First Name if available

c.   client Last Name if available

d.    ﬁrst call date

e.    average calls duration for that client if available

f.    client phone number

 Warning : You can see that not all clients are referenced in the calls table, and vice versa !

 Indeed :

●  Some clients are present in the client_crm database but we have not tracked properly their calls, so they are not present in the calls database

(Tips : If a client's ﬁrst call date is unavailable, we can use the client record creation date.)

●   Some prospects (potential clients) contacted us by phone but the agent who took the call did not register them properly in the client_crm database (you will not be able to ﬁnd the ID, First Name and LastName for all the contacts)

SELECT cl.id, cl.FirstName, cl.LastName, IF(min(ca.date) IS NULL, cl.CreationDate, min(ca.date)) as first_call_date, IF(cl.PhoneNumber IS NULL, CONVERT(ca.incoming_number, CHAR), cl.PhoneNumber) as phone_number
FROM calls ca
OUTER JOIN clients_crm cl ON CONVERT(ca.incoming_number, CHAR) = cl.PhoneNumber
GROUP BY ca.incoming_number;

 3. In a phone-based model, one of the main difﬁculties is to properly track clients. As mentioned before, you can see that not all clients are referenced in the calls table, and vice versa: To deal with this difﬁculty, management suggests to track the transformation rate at a daily bucket. In other words, for each day, we want to know the ratio of the number of clients with a record in our CRM out of the number of calls received that same day.Can you provide the report with ALL clients and ALL calls taken into account ? With the following structure:

a.    day

b.    number of calls

c.   number of clients

d.    transformation rate (clients / calls)

SELECT DAY(ca.date) as day, count(ca.id) as num_calls, count(cl.id) as num_clients, sum(IF(cl.id IS NULL, 0, 1))/count(ca.id) as transformation_rate
FROM calls ca
LEFT JOIN clients_crm cl ON CONVERT(ca.incoming_number, CHAR) = cl.PhoneNumber
GROUP BY day;

 4. Looking more deeply at the client_crm table, it seems that we have clients with identical phone numbers! Can you suggest a query to get rid of those « duplicates »? (we want to keep only one unique client record per phone number)

 Tips: Note that incoming_number in the calls table, and PhoneNumber in the clients_crm table may not have the same data types and format.

 SELECT id, FirstName, LastName, PhoneNumber, CreationDate FROM
 (SELECT id, FirstName, LastName, PhoneNumber, CreationDate, row_number() over (partition by id order by CreationDate) as rn
 FROM clients_crm)
 WHERE rn = 1;